In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt

---
Get the centroids of sa2

In [2]:
GCCSA_2016 = gpd.read_file('../data/raw/map polygons/1270055001_gccsa_2016_aust_shape/GCCSA_2016_AUST.shp')
GCCSA_2016 = GCCSA_2016.to_crs('EPSG:7855')
Greater_Melbourne_2016 = GCCSA_2016.iloc[3]['geometry']

In [3]:
SA2_2016 = gpd.read_file('../data/raw/map polygons/1270055001_sa2_2016_aust_shape/SA2_2016_AUST.shp')
SA2_2016 = SA2_2016.to_crs('EPSG:7855')
SA2_GreaterMelbourne_2016 = SA2_2016[SA2_2016.within(Greater_Melbourne_2016)]
SA2_GreaterMelbourne_2016['centre'] = SA2_GreaterMelbourne_2016['geometry'].centroid.apply(lambda x: Point(x.x, x.y))
SA2_GreaterMelbourne_2016_centroid = gpd.GeoDataFrame(SA2_GreaterMelbourne_2016, geometry='centre')
# SA2_GreaterMelbourne_2016_centroid[['SA2_NAME16', 'centre']].to_file('../data/curated/stations/GreaterMelbourneCentroid2016.shp')

/Applications/anaconda3/envs/spanalytics/lib/python3.11/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [12]:
GCCSA_2021 = gpd.read_file('../data/raw/map polygons/GCCSA_2021_AUST_SHP_GDA2020/GCCSA_2021_AUST_GDA2020.shp')
GCCSA_2021 = GCCSA_2021.to_crs('EPSG:7855')
Greater_Melbourne_2021 = GCCSA_2021.iloc[4]['geometry']

In [13]:
SA2_2021 = gpd.read_file('../data/raw/map polygons/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')
SA2_2021 = SA2_2021.to_crs('EPSG:7855')
SA2_GreaterMelbourne_2021 = SA2_2021[SA2_2021.within(Greater_Melbourne_2021)]
SA2_GreaterMelbourne_2021['centre'] = SA2_GreaterMelbourne_2021['geometry'].centroid.apply(lambda x: Point(x.x, x.y))
SA2_GreaterMelbourne_2021_centroid = gpd.GeoDataFrame(SA2_GreaterMelbourne_2021, geometry='centre')
# SA2_GreaterMelbourne_2021_centroid[['SA2_NAME21', 'centre']].to_file('../data/curated/stations/GreaterMelbourneCentroid2021.shp')

/Applications/anaconda3/envs/spanalytics/lib/python3.11/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


---

In [88]:
mean_daily_train_demand_precovid = gpd.read_file('../data/curated/train_demand/mean_daily_trains_demand_pre_covid.gpkg')
mean_daily_train_demand_postcovid = gpd.read_file('../data/curated/train_demand/mean_daily_trains_demand_post_covid.gpkg')
mean_daily_train_demand_precovid = mean_daily_train_demand_precovid.to_crs('EPSG:7855')
mean_daily_train_demand_postcovid = mean_daily_train_demand_postcovid.to_crs('EPSG:7855')

In [89]:
# join up 
train_stations_in_sa2_precovid = gpd.sjoin(mean_daily_train_demand_precovid, SA2_2016, how = 'left', predicate='within')
train_stations_in_sa2_postcovid = gpd.sjoin(mean_daily_train_demand_postcovid, SA2_2021, how='left', predicate='within')

# merge the data to get the polygons because sjoins take out the right's geometry (each gdf should only have 1 geometry)
train_stations_in_sa2_precovid = train_stations_in_sa2_precovid.merge(SA2_2016, left_on='SA2_MAIN16', right_on='SA2_MAIN16')
train_stations_in_sa2_postcovid = train_stations_in_sa2_postcovid.merge(SA2_2021, left_on='SA2_CODE21', right_on='SA2_CODE21')

train_stations_in_sa2_precovid = train_stations_in_sa2_precovid[['Station_Name', 'Passenger_Boardings', 'Passenger_Alightings',
       'Total_Demand', 'log_Passenger_Boardings', 'log_Passenger_Alightings',
       'log_Total_Demand', 'geometry_y', 'SA2_NAME16_x']]
train_stations_in_sa2_precovid = train_stations_in_sa2_precovid.rename(columns={'geometry_y': 'geometry', 'SA2_NAME16_x': 'SA2_NAME16'})
train_stations_in_sa2_postcovid = train_stations_in_sa2_postcovid[['Station_Name', 'Passenger_Boardings', 'Passenger_Alightings',
       'Total_Demand', 'log_Passenger_Boardings', 'log_Passenger_Alightings',
       'log_Total_Demand', 'geometry_y', 'SA2_NAME21_x']]
train_stations_in_sa2_postcovid = train_stations_in_sa2_postcovid.rename(columns={'geometry_y': 'geometry', 'SA2_NAME21_x': 'SA2_NAME21'})

train_stations_in_sa2_precovid = gpd.GeoDataFrame(train_stations_in_sa2_precovid, geometry='geometry')
train_stations_in_sa2_postcovid = gpd.GeoDataFrame(train_stations_in_sa2_postcovid, geometry='geometry')

train_stations_in_sa2_precovid.to_file('../data/curated/train_demand/train_stations_in_sa2_precovid.shp')
train_stations_in_sa2_postcovid.to_file('../data/curated/train_demand/train_stations_in_sa2_postcovid.shp')

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_86236/2669080601.py:21: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  train_stations_in_sa2_precovid.to_file('../data/curated/stations/train_stations_in_sa2_precovid.shp')
/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_86236/2669080601.py:22: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  train_stations_in_sa2_postcovid.to_file('../data/curated/stations/train_stations_in_sa2_postcovid.shp')


---
Concat list of suburbs w/out stations and stations

In [100]:
mean_daily_train_demand_precovid_stations = mean_daily_train_demand_precovid[['Station_Name', 'geometry']]
mean_daily_train_demand_postcovid_stations = mean_daily_train_demand_postcovid[['Station_Name', 'geometry']]
mean_daily_train_demand_precovid_stations['point_type'] = 'station'
mean_daily_train_demand_postcovid_stations['point_type'] = 'station'
mean_daily_train_demand_precovid_stations = mean_daily_train_demand_precovid_stations.rename(columns={'Station_Name': 'Point Name'})
mean_daily_train_demand_postcovid_stations = mean_daily_train_demand_postcovid_stations.rename(columns={'Station_Name': 'Point Name'})

SA2_GreaterMelbourne_2016_centroid_suburbs_only = SA2_GreaterMelbourne_2016_centroid[['SA2_NAME16', 'centre']]
SA2_GreaterMelbourne_2016_centroid_suburbs_only['point_type'] = 'suburb'
SA2_GreaterMelbourne_2016_centroid_suburbs_only = SA2_GreaterMelbourne_2016_centroid_suburbs_only.rename(columns={'SA2_NAME16': 'Point Name', 'centre': 'geometry'})

SA2_GreaterMelbourne_2021_centroid_suburbs_only = SA2_GreaterMelbourne_2021_centroid[['SA2_NAME21', 'centre']]
SA2_GreaterMelbourne_2021_centroid_suburbs_only['point_type'] = 'suburb'
SA2_GreaterMelbourne_2021_centroid_suburbs_only = SA2_GreaterMelbourne_2021_centroid_suburbs_only.rename(columns={'SA2_NAME21': 'Point Name', 'centre': 'geometry'})

all_points_precovid = pd.concat([mean_daily_train_demand_precovid_stations, SA2_GreaterMelbourne_2016_centroid_suburbs_only])
all_points_postcovid = pd.concat([mean_daily_train_demand_postcovid_stations, SA2_GreaterMelbourne_2021_centroid_suburbs_only])

/Applications/anaconda3/envs/spanalytics/lib/python3.11/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Applications/anaconda3/envs/spanalytics/lib/python3.11/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Applications/anaconda3/envs/spanalytics/lib/python3.11/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be s

In [103]:
suburbs_with_stations_2016 = train_stations_in_sa2_precovid['SA2_NAME16'].unique()
suburbs_with_stations_2021 = train_stations_in_sa2_postcovid['SA2_NAME21'].unique()
all_points_precovid = all_points_precovid[~(all_points_precovid['Point Name'].isin(suburbs_with_stations_2016)) | (all_points_precovid['point_type'] == 'station')]
all_points_postcovid = all_points_postcovid[~(all_points_postcovid['Point Name'].isin(suburbs_with_stations_2021)) | (all_points_postcovid['point_type'] == 'station')]

In [106]:
all_points_precovid.to_file('../data/curated/stations/all_points_precovid_(SA2 without stations).shp')
all_points_postcovid.to_file('../data/curated/stations/all_points_postcovid_(SA2 without stations).shp')